# Bert Centralised Learning

Set the module directory to import python files (RUN JUST ONCE)

In [3]:

import os
os.chdir('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your project
import sys
sys.path.append('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your models directory
print(sys.path)

%load_ext autoreload
%autoreload 2



['/home/victor/_bcfl/fabric-federated-learning/federated-learning/client/notebooks', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python311.zip', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/lib-dynload', '', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/site-packages', '/home/victor/_bcfl/fabric-federated-learning/federated-learning']


In [4]:
import torch
print(f"Cuda available: {torch.cuda.is_available()}")
# Get the name of the CUDA device
print(torch.cuda.get_device_name(0))

try:
    print(
        f"major and minor cuda capability of the device: {torch.cuda.get_device_capability()}")
except Exception:
    print("No Cuda available")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Check if CUDA is available and set the default tensor type to CUDA
print('Using device: %s' % device)
if torch.cuda.is_available():
    torch.set_default_device('cuda')
    print("Cuda set as default device")
else:
    torch.set_default_device('cpu')
    print("Cuda not available, CPU set as default device")

Cuda available: False


/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400430266/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [5]:
# NVIDIA drivers not working
torch.set_default_device('cpu')
device = torch.device('cpu')

### Initialize variables

In [6]:
from client.model.bert_tiny import get_bert_tiny_tokenizer

MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
EVAL_BATCH_SIZE = 2
TEST_BATCH_SIZE = 8
NUM_EPOCHS = 10
LEARNING_RATE = 1e-05
tokenizer = get_bert_tiny_tokenizer()
root = 'client/data/datasets/dep1_cleaned.csv'



/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading Reddit Depression Dataloaders

In [7]:
from client.data.reddit_dep import get_reddit_dep_dataloaders


train_loader, eval_loader, test_loader = get_reddit_dep_dataloaders(root, tokenizer, 
                                                                    train_size=0.5,
                                                                    eval_size=0,
                                                                    test_size=0.5,
                                                                    train_batch_size=TRAIN_BATCH_SIZE, 
                                                                    eval_batch_size=EVAL_BATCH_SIZE, 
                                                                    test_batch_size=TEST_BATCH_SIZE, 
                                                                    max_len=MAX_LEN, 
                                                                    device=device)


### Creating Bert Tiny Model

In [9]:
from client.model.bert_tiny import get_bert_tiny_model

bert_tiny = get_bert_tiny_model(device=device)
print(bert_tiny)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, e

### Loading Transformers Trainer

In [23]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
from client.data.reddit_dep import get_reddit_dep_datasets

train_dataset, eval_dataset, test_dataset = get_reddit_dep_datasets(root, tokenizer, 0.2, 0.2, 0.6, MAX_LEN)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metric = evaluate.load("accuracy")
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="client/models/bert-tiny/", evaluation_strategy="epoch")


trainer = Trainer(
    model=bert_tiny,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

### Train model with Trainer 

In [1]:
trainer.train()

NameError: name 'trainer' is not defined

### Test the trained model

In [2]:
from client.test import test_text_class

# Test the model
test_text_class(bert_tiny, test_loader, device=device)

ModuleNotFoundError: No module named 'client'